# Gaze Interception
This notebook takes in three files taken from the Pupil Recordings you have exported in Pupil Player. These are:
- `gaze_positions.csv` (contains raw data in regards to the gaze made throughout the recording)
- `info.player.json` (contains system and sync time used to format the recording timestamps)

In [40]:
# Imports
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import json
import warnings

matplotlib.use('TkAgg')
warnings.filterwarnings("ignore", category=RuntimeWarning)

# File Paths
info_player_filePath = 'source/info.player.json'
gaze_csv_filePath = 'source/gaze_positions.csv'
annotations_filepath = 'source/annotations.csv'
fixation_filepath = 'source/fixations.csv'

# Creating the DataFrame for the data

Reading the `info.player` JSON to retrieve `start_time_synced_s` and `start_time_system_s`. This is used to format the timestamp correctly to indicate time throughout the experiment:

In [41]:
def obtain_offset(filepath):
    with open(info_player_filePath, 'r') as file:
        data = json.load(file)  
    return data.get('start_time_system_s') - data.get('start_time_synced_s')

Now, reading the gaze_position.csv file to obtain base data, formating time, calculating smoothed positions, and angular distance and velocity of each timestamp. And removing data that contains infinte or NaN values.

In [42]:
# Calculate angular distance given Cartesian coordinates (x, y)
def calculate_angular_distance(x, y):
    return np.arctan2(y, x)

# Calculate velocity given angular distance and corresponding timestamps.
def calculate_velocity(angular_distance, timestamp):
    time_diff = np.diff(timestamp)
    angular_distance_diff = np.diff(angular_distance)
    velocity = angular_distance_diff / time_diff
    return np.concatenate(([np.nan], velocity))


# Applies Rolling Medium over the input field, and exports that into the output field
def smooth_data(df, input, output, window_size=0):
    df[output] = df[input].rolling(window=window_size).median()
    return df

def obtain_gaze_data(filepath: str, window=10, offset=0):
    csv_df = pd.read_csv(filepath)
    results_df = pd.DataFrame(columns=['gaze_timestamp',
                                       'time',
                                       'norm_pos_x', 
                                       'norm_pos_y', 
                                       'angular_distance',
                                       'angular_velocity',
                                       'movement_type',
                                       'smoothed_norm_pos_x', 
                                       'smoothed_norm_pos_y',
                                       'smoothed_angular_distance',
                                       'smoothed_angular_velocity',
                                       'smoothed_movement_type'])

    # Copy the data from csv of necessary fields
    results_df['gaze_timestamp'] = csv_df['gaze_timestamp']
    results_df['norm_pos_x'] = csv_df['norm_pos_x']
    results_df['norm_pos_y'] = csv_df['norm_pos_y']
    results_df['movement_type'] = 'None'
    results_df['smoothed_movement_type'] = 'None'
    
    # Populating the time field by adding the offset and then subtracting the minimum time to start from 0
    results_df['time'] = results_df['gaze_timestamp'] + offset
    results_df['time'] -= results_df['time'].min()

    # Using rolling mean to smooth the data and to rmeove as many extreme outliers that has been missed by Pupil Export
    results_df['smoothed_norm_pos_x'] = results_df['norm_pos_x'].rolling(window).median()
    results_df['smoothed_norm_pos_y'] = results_df['norm_pos_y'].rolling(window).median()

    # Calculating the angular distance for each x and y position for both smoothed and un-smoothed
    results_df['angular_distance'] = calculate_angular_distance(results_df['norm_pos_x'], results_df['norm_pos_y'])
    results_df['smoothed_angular_distance'] = calculate_angular_distance(results_df['smoothed_norm_pos_x'], results_df['smoothed_norm_pos_y'])

    # Calculating the angular velocity for both smooth and un-smoothed angular distances over time
    results_df['angular_velocity'] = calculate_velocity(results_df['angular_distance'], results_df['time'])
    results_df['smoothed_angular_velocity'] = calculate_velocity(results_df['smoothed_angular_distance'], results_df['time'])

    # Dropping NaN records and the `gaze_timestamp` field
    results_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    results_df.dropna(inplace=True)
    results_df = results_df.drop('gaze_timestamp', axis=1) # Remove the `gaze_timestamp` as that is not needed anymore

    return results_df

window_size=10
offset = obtain_offset(info_player_filePath)
gaze_df = obtain_gaze_data(gaze_csv_filePath, window=window_size, offset=offset)
print(f'There is a total of {len(gaze_df)} gaze positions')


There is a total of 8047 gaze positions


### Predicting Smooth Pursuit
Smooth pursuit occurs when the eyes tracks an moving object. There is roughly a constant velocity as the angular distance changes slowly, whereas Saccades are almost instant. We choose regions of the line where the change in velocity is under a threshold, and the change of angular distance is small enough to indicate the eyes moving between positions slowly.

In [43]:
def predict_smooth_pursuit(df, threshold_velocity=0.2, angular_distance_threshold=2e-4):
    pursuit_regions_velocity = np.abs(np.gradient(df['angular_velocity'])) < threshold_velocity
    pursuit_regions_distance = np.abs(np.gradient(df['angular_distance'])) > angular_distance_threshold
    pursuit_regions = (pursuit_regions_velocity) & (pursuit_regions_distance)
    df['movement_type'] = np.where((df['movement_type'] == 'None') & pursuit_regions, 'Smooth Pursuit', df['movement_type'])
    
    pursuit_regions_smoothed_velocity = np.abs(np.gradient(df['smoothed_angular_velocity'])) < threshold_velocity
    pursuit_regions_smoothed_distance = np.abs(np.gradient(df['smoothed_angular_distance'])) > angular_distance_threshold
    pursuit_regions_smoothed = (pursuit_regions_smoothed_velocity) & (pursuit_regions_smoothed_distance)
    df['smoothed_movement_type'] = np.where((df['smoothed_movement_type'] == 'None') & pursuit_regions_smoothed, 'Smooth Pursuit', df['smoothed_movement_type'])

predict_smooth_pursuit(gaze_df)
smooth_pursuit = gaze_df[(gaze_df['movement_type'] == 'Smooth Pursuit') | (gaze_df['smoothed_movement_type'] == 'Smooth Pursuit')]
print(f'Total Smooth Pursuit Movements: {len(smooth_pursuit)}')

Total Smooth Pursuit Movements: 6002


### Predicting Saccades
Based on the research from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1190820/pdf/jphysiol00502-0164.pdf, Saccadic Movements have a stereotypical velocity graph where large peaks indicates a fast movement between two points (saccades). These are almost like straight lines between points. We can say that if the velocity between two  points exceeds a threshold, then this is most possibly a saccadic movement.

In [44]:
def predict_saccades(df, threshold_velocity=0.6):
    saccade_regions_high = df['angular_velocity'] > threshold_velocity
    accade_regions_low = df['angular_velocity'] < -threshold_velocity
    saccade_regions = saccade_regions_high | accade_regions_low
    df['movement_type'] = np.where(saccade_regions, 'Saccades', 'None')
    
    saccade_regions_high = df['smoothed_angular_velocity'] > threshold_velocity
    accade_regions_low = df['smoothed_angular_velocity'] < -threshold_velocity
    saccade_regions = saccade_regions_high | accade_regions_low
    df['smoothed_movement_type'] = np.where((df['smoothed_movement_type'].isin(['None', 'Smooth Pursuit'])) & saccade_regions, 'Saccades', df['smoothed_movement_type'])

predict_saccades(gaze_df)
saccades = gaze_df[(gaze_df['movement_type'] == 'Saccades') | (gaze_df['smoothed_movement_type'] == 'Saccades')]
print(f'Total Saccadic Movements: {len(saccades)}')

Total Saccadic Movements: 2602


### Predicting Fixation
Fixations are defined as gaze remain fixed around a stationary point. It is not directly fixed but the user gazes around the same point for a long period of time.

Pupil Player provides another csv file called `fixations` which identifies the timestamp of when a fixation occurs, and how long it can take. The function `plot_fixation_graph` takes regions starting from whatever second the fixation starts up until the end of the fixation indicated by by the `duration` field.

In [45]:
def predict_fixation(df, offset, time_offset=0.15, filepath='fixations.csv'):
    fixations_df = pd.read_csv(filepath)
    fixations_df['time'] = fixations_df['start_timestamp'] + offset
    fixations_df['time'] -= fixations_df['time'].min()

    for _, fixation in fixations_df.iterrows():
        start_time = fixation['time']
        duration = (fixation['duration'] / 1000.0) - time_offset  # Convert into seconds

        # Mark rows within the fixation region as 'Fixation' in both columns
        fixation_regions = (df['time'] >= start_time) & (df['time'] <= start_time + duration)
        df.loc[fixation_regions, 'movement_type'] = 'Fixation'
        df.loc[fixation_regions, 'smoothed_movement_type'] = 'Fixation'

predict_fixation(gaze_df, offset, filepath=fixation_filepath)
fixation = gaze_df[(gaze_df['movement_type'] == 'Fixation') | (gaze_df['smoothed_movement_type'] == 'Fixation')]
print(f'Total Fixation Movements: {len(fixation)}')

Total Fixation Movements: 1039


### Exporting Data
To be used for later or with the Visualier, this exports the dataframe and an json containing the offset for annotations

In [46]:
# Allows the dataframe to be exported as a csv file of a given name as well as a JSON file that stores the offset if the graph was to be used again
def export_gaze_data(df, offset, filename='results'):
    df.to_csv(f'{filename}.csv', index=False)
    with open('offset.json', 'w') as json_file:
        json.dump({'offset':offset}, json_file)

filename = 'test'
export_gaze_data(gaze_df, offset, filename) # Exporting the dataframe as a csv file